In [1]:
import os
import glob
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/project/RetinaNet

/content/drive/MyDrive/project/RetinaNet


In [ ]:
path = '/content/drive/MyDrive/project/RetinaNet/food'
img_folder = os.listdir(path)

img_folder.remove('category.txt')
img_folder


['1', '17', '18', '19', '20', '27']

### class_mapping.csv 만들기
클래스이름과 레이블이 있는 파일 만들어짐

In [ ]:
# class_mapping.csv 만들기
class_data = pd.read_csv('/content/drive/MyDrive/project/RetinaNet/food/category.txt', sep = "\t")

class_list = class_data.loc[class_data['id'].isin(img_folder[:-1])]

class_list.iloc[:, 0] = class_list.iloc[:, 1]
class_list.iloc[:, 1] = np.arange(0, len(class_list))

class_list.to_csv('class_mapping.csv', index=None, header=None)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [ ]:
class_data

,id,name
0,1,rice
1,2,eels on rice
2,3,pilaf
3,4,chicken-'n'-egg on rice
4,5,pork cutlet on rice
...,...,...
95,96,dipping noodles
96,97,hot dog
97,98,french fries
98,99,mixed rice


In [ ]:
class_list

In [ ]:
# 각 폴더에 있는 이미지 개수 출력
!ls food/1 | wc -l
!ls food/17 | wc -l
!ls food/18 | wc -l
!ls food/19 | wc -l
!ls food/20 | wc -l
!ls food/27 | wc -l

621
234
135
164
153
152


### train : test = 8 : 2
분리하여 경로 따로 저장하기

In [ ]:
import random
train_dir = np.array([])
test_dir = np.array([])

for i in img_folder:
    # 데이터 불러오기 
    # 랜덤으로 섞은 후 120 개만
    img_path = '/content/drive/MyDrive/project/RetinaNet/food/' + i
    img_glob = glob.glob(os.path.join(img_path, '*.jpg'))
    random.seed(10)
    random.shuffle(img_glob)
    img_glob = img_glob[:120]

    len_split = int(len(img_glob) * 0.8)
    # print(len_split)
    train_dir = np.append(train_dir, img_glob[:len_split])
    test_dir = np.append(test_dir, img_glob[len_split:])
    # print(len(train_dir))
    
print('train 개수:', len(train_dir))
print('test 개수:', len(test_dir))

96
192
288
384
480
576
train 개수: 576
test 개수: 144


### train annotation 파일 만들기
img 경로, x1, y1, x2, y2, ClassName으로 구성

In [ ]:
df = pd.DataFrame(columns=['img', 'x1', 'y1', 'x2', 'y2'], dtype='object')
class_df = pd.DataFrame(columns=['className'])

for idx, i in enumerate(train_dir):
    img_dir = os.path.split(i)[0]
    bbox_dir = img_dir + '/bb_info.txt'
    bbox_df = pd.read_csv(bbox_dir, sep = " ")
    img_name = os.path.split(i)[1].split('.')[0] # 1.jpg -> 1

    roi_bbox = bbox_df.loc[bbox_df['img'] == int(img_name)]
    df.loc[idx] = roi_bbox.values[0]

    file_name = os.path.split(i)[0].split('/')[-1] # 폴더명 ( 1, 17, ..)
    class_name = class_data['name'][class_data['id'] == int(file_name)]
    class_df.loc[idx] = class_name.values[0]

df['img'] = train_dir
fin_df = pd.concat([df, class_df], axis=1)

In [ ]:
fin_df

In [ ]:
fin_df.to_csv('annotation.csv', header=False, index=False)

### test annotation 파일 만들기

In [ ]:
df = pd.DataFrame(columns=['img', 'x1', 'y1', 'x2', 'y2'], dtype='object')
class_df = pd.DataFrame(columns=['className'])

for idx, i in enumerate(test_dir):
    img_dir = os.path.split(i)[0]
    bbox_dir = img_dir + '/bb_info.txt'
    bbox_df = pd.read_csv(bbox_dir, sep = " ")
    img_name = os.path.split(i)[1].split('.')[0] # 1.jpg -> 1

    roi_bbox = bbox_df.loc[bbox_df['img'] == int(img_name)]
    df.loc[idx] = roi_bbox.values[0]

    file_name = os.path.split(i)[0].split('/')[-1] # 폴더명 ( 1, 17, ..)
    class_name = class_data['name'][class_data['id'] == int(file_name)]
    class_df.loc[idx] = class_name.values[0]

df['img'] = test_dir
fin_df = pd.concat([df, class_df], axis=1)

In [ ]:
fin_df.to_csv('test_annotation.csv', header=False, index=False)

In [ ]:
fin_df

,img,x1,y1,x2,y2,className
0,/content/drive/MyDrive/project/RetinaNet/food/...,12,230,233,399,rice
1,/content/drive/MyDrive/project/RetinaNet/food/...,49,307,340,560,rice
2,/content/drive/MyDrive/project/RetinaNet/food/...,28,232,308,442,rice
3,/content/drive/MyDrive/project/RetinaNet/food/...,458,170,661,376,rice
4,/content/drive/MyDrive/project/RetinaNet/food/...,69,12,462,355,rice
...,...,...,...,...,...,...
139,/content/drive/MyDrive/project/RetinaNet/food/...,2,86,366,457,spaghetti
140,/content/drive/MyDrive/project/RetinaNet/food/...,0,33,360,411,spaghetti
141,/content/drive/MyDrive/project/RetinaNet/food/...,10,84,500,284,spaghetti
142,/content/drive/MyDrive/project/RetinaNet/food/...,274,34,597,433,spaghetti


In [ ]:
class_df = pd.DataFrame(columns=['className'])

for idx, i in enumerate(train_dir[:100]):
    file_name = os.path.split(i)[0].split('/')[-1] # 폴더명 ( 1, 17, ..)
    class_name = class_data['name'][class_data['id'] == int(file_name)]
    # print(class_name)
    # class_df.append(class_name)
    class_df.loc[idx] = class_name.values[0]
    # class_df['className'] = class_name.values[0]


In [ ]:
/opt/homebrew/Cellar/opencv/4.5.3_1

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [4]:
cd /content/drive/MyDrive/project/RetinaNet/keras-retinanet

/content/drive/MyDrive/project/RetinaNet/keras-retinanet


In [5]:
!python setup.py install build_ext --inplace

running install
running bdist_egg
running egg_info
writing keras_retinanet.egg-info/PKG-INFO
writing dependency_links to keras_retinanet.egg-info/dependency_links.txt
writing entry points to keras_retinanet.egg-info/entry_points.txt
writing requirements to keras_retinanet.egg-info/requires.txt
writing top-level names to keras_retinanet.egg-info/top_level.txt
reading manifest file 'keras_retinanet.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'keras_retinanet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.7/keras_retinanet/utils/compute_overlap.cpython-37m-x86_64-linux-gnu.so -> keras_retinanet/utils
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/keras_retinanet
copying build/lib.linux-x86_64-3.7/keras_retinanet/losses.py -> build/bdi

In [ ]:
a = pd.read_csv('/content/drive/MyDrive/project/RetinaNet/train.csv', header=None)
len(a)

1000

In [3]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y

!pip install tensorflow==2.3.0
!pip install keras==2.4

Found existing installation: Keras 2.4.3
Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
Found existing installation: keras-nightly 2.5.0.dev2021032900
Uninstalling keras-nightly-2.5.0.dev2021032900:
  Successfully uninstalled keras-nightly-2.5.0.dev2021032900
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 320.4 MB 45 kB/s 
     |████████████████████████████████| 2.9 MB 40.5 MB/s 
     |████████████████████████████████| 20.1 MB 89.1 MB/s 
     |████████████████████████████████| 459 kB 68.7 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
  Attempting uninstall: numpy
    Found existi

     |████████████████████████████████| 170 kB 8.3 MB/s 


In [7]:
!python keras_retinanet/bin/train.py --steps 800 csv /content/drive/MyDrive/project/RetinaNet/train.csv /content/drive/MyDrive/project/RetinaNet/classMapping.csv


225/800 [=======>......................] - ETA: 3:24 - loss: 1.4064 - regression_loss: 0.9382 - classification_loss: 0.4682Traceback (most recent call last):
  File "keras_retinanet/bin/train.py", line 554, in <module>
    main()
  File "keras_retinanet/bin/train.py", line 549, in main
    initial_epoch=args.initial_epoch
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 1829, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 1103, in fit
    callbacks.on_train_batch_end(end_step, logs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/py

In [ ]:
!python keras_retinanet/bin/train.py --steps 800 csv /content/drive/MyDrive/project/RetinaNet/train.csv /content/drive/MyDrive/project/RetinaNet/classMapping.csv --val-annotations /content/drive/MyDrive/project/RetinaNet/test.csv

2021-08-04 12:06:36.825803: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
Creating model, this may take a second...
2021-08-04 12:06:39.694048: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-04 12:06:39.729293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-04 12:06:39.729888: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-08-04 12:06:39.729930: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-

In [ ]:
!python keras_retinanet/bin/convert_model.py snapshots/resnet50_csv_30.h5 /content/drive/MyDrive/project/RetinaNet/keras-retinanet/model/retinanet_model.h5

2021-08-04 14:54:36.865681: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-04 14:54:42.208781: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-04 14:54:42.245301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-04 14:54:42.245993: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-08-04 14:54:42.246038: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-04 14:54:42.258113: I tensorflow/stream_executor/platform/default

In [ ]:
!python keras_retinanet/bin/evaluate.py csv /content/drive/MyDrive/project/RetinaNet/test.csv /content/drive/MyDrive/project/RetinaNet/classMapping.csv /content/drive/MyDrive/project/RetinaNet/keras-retinanet/model/retinanet_model.h5

2021-08-04 14:55:19.451903: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
Loading model, this may take a second...
2021-08-04 14:55:22.953766: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-04 14:55:22.984738: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-04 14:55:22.985356: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-08-04 14:55:22.985398: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-0

In [ ]:
!python /content/drive/MyDrive/project/RetinaNet/keras-retinanet/testDetector.py

2021-08-04 15:04:30.444599: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.

2021-08-04 15:04:32.012713: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-04 15:04:32.018772: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2199995000 Hz
2021-08-04 15:04:32.018974: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562279ae0a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-04 15:04:32.019006: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-08-04 15:04:32.021073: I tensorflow/stream_